<h2>2. Импорт библиотек</h2>

In [6]:
#импорт библиотек
import glob
import os
import pandas as pd
import numpy as np
import json
import docx
from bs4 import BeautifulSoup as bs
import requests
from tqdm.auto import tqdm, trange
from pymystem3 import Mystem
from nltk.stem.snowball import SnowballStemmer 
from nltk.tokenize import word_tokenize
from nltk.stem import *
from nltk.corpus import stopwords
import string
import re
import pyLDAvis
import pyLDAvis.lda_model
import pymorphy2
import warnings
warnings.filterwarnings("ignore")
import pandas as pd
import numpy as np
from tqdm.auto import tqdm, trange
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.linear_model import SGDClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve
import seaborn as sns
from matplotlib import pyplot as plt
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import RepeatedKFold
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from imblearn.over_sampling import SMOTE
from sklearn.preprocessing import StandardScaler
import joblib
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

<h2>2.1 Построение модели классификации</h2>

<h3>2.1.1 Разбиение выборки на обучающую и тестовую</h3>

In [9]:
df = pd.read_csv('dataset.csv')

In [10]:
df.head()

,Unnamed: 0,namecompany,text,date,rating,industries,text_stem,text_sw,text_lemm,text_ps,keywords,bigrams,trigrams,target
0,0,Cybersport,7 декабря состоялась церемония награждения лау...,8 декабря 09:30,94.926255,"['Веб-разработка', 'Дизайн и юзабилити', 'Рекл...",декабр состоя церемон награжден лауреат прем р...,декабря состоялась церемония награждения лауре...,декабрь состояться церемония награждение лауре...,"['(декабрь, NOUN)', '(состояться, INFN)', '(це...","{'agency': 0.027513674254937545, 'balance': 0....","{'agency проект': 0.047036043419179864, 'balan...",{'agency проект вебинклюзия': 0.05006261743217...,Нет номинации
1,1,ESforce Holding,7 декабря состоялась церемония награждения лау...,8 декабря 09:30,94.926255,"['Веб-разработка', 'Дизайн и юзабилити', 'Рекл...",декабр состоя церемон награжден лауреат прем р...,декабря состоялась церемония награждения лауре...,декабрь состояться церемония награждение лауре...,"['(декабрь, NOUN)', '(состояться, INFN)', '(це...","{'agency': 0.027513674254937545, 'balance': 0....","{'agency проект': 0.047036043419179864, 'balan...",{'agency проект вебинклюзия': 0.05006261743217...,Игровая индустрия и киберспорт
2,2,ESforce Holding,Из совместного исследования издателя компьютер...,29 июля 13:50,94.926255,"['Веб-разработка', 'Дизайн и юзабилити', 'Рекл...",совместн исследован издател компьютерн игр war...,совместного исследования издателя компьютерных...,совместный исследование издатель компьютерный ...,"['(совместный, ADJF)', '(исследование, NOUN)',...","{'and': 0.03980937143290165, 'assassin': 0.039...","{'and zombies': 0.05590169943749474, 'assassin...","{'and zombies серия': 0.05986843400892496, 'as...",Игровая индустрия и киберспорт
3,3,ESforce Holding,По статистике The Esports Observer за I полови...,9 июля 09:33,94.926255,"['Веб-разработка', 'Дизайн и юзабилити', 'Рекл...",статистик the esports observer i половин год в...,статистике the esports observer i половину год...,статистика the esports observer i половина год...,"['(статистика, NOUN)', '(the, None)', '(esport...","{'apex': 0.0331861655799986, 'auto': 0.0331861...","{'apex legends': 0.05598925109558543, 'auto мл...","{'apex legends млн': 0.05892556509887897, 'aut...",Игровая индустрия и киберспорт
4,4,ESforce Holding,Компании SuperData Research и PayPal провели с...,11 апреля 12:43,94.926255,"['Веб-разработка', 'Дизайн и юзабилити', 'Рекл...",компан superdata research paypal провел совмес...,компании superdata research paypal провели сов...,компания superdata research paypal проводить с...,"['(компания, NOUN)', '(superdata, None)', '(re...","{'are': 0.02857142857142857, 'arena': 0.028571...","{'are the': 0.04490132550669373, 'arena moscow...","{'are the world': 0.047836487323493986, 'arena...",Игровая индустрия и киберспорт


In [11]:
X = df['text_lemm']
y = df['target']

In [12]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size =0.3, random_state = 42)

In [13]:
my_tags = df['target'].unique()

In [14]:
my_tags

array(['Нет номинации', 'Игровая индустрия и киберспорт',
       'Наука, технологии и инновации', 'Креативные индустрии',
       'Образование и Кадры', 'Экономика и Бизнес'], dtype=object)

<h3>2.1.2 Классификация Naive Bayes</h3>

In [15]:
nb = Pipeline([('vect', CountVectorizer()),
              ('tfidf', TfidfTransformer()),
              ('clf', MultinomialNB()),])

In [16]:
%%time
# Обучение модели
nb.fit(X_train, y_train)

CPU times: total: 1.62 s
Wall time: 1.68 s


Pipeline(steps=[('vect', CountVectorizer()), ('tfidf', TfidfTransformer()),
                ('clf', MultinomialNB())])

In [17]:
%%time
y_pred = nb.predict(X_test)

CPU times: total: 641 ms
Wall time: 692 ms


In [18]:
# Результаты классификации naive bayes
from sklearn.metrics import accuracy_score

print('accuracy %s' % accuracy_score(y_pred, y_test))
print(classification_report(y_test, y_pred, target_names=my_tags))

accuracy 0.43485342019543977
                                precision    recall  f1-score   support

                 Нет номинации       0.00      0.00      0.00         2
Игровая индустрия и киберспорт       0.00      0.00      0.00        71
 Наука, технологии и инновации       0.43      1.00      0.60       257
          Креативные индустрии       1.00      0.08      0.15       120
           Образование и Кадры       0.00      0.00      0.00       118
            Экономика и Бизнес       0.00      0.00      0.00        46

                      accuracy                           0.43       614
                     macro avg       0.24      0.18      0.13       614
                  weighted avg       0.37      0.43      0.28       614

